In [1]:
%run ../../Utils/yp_utils.py

# Initial setup

In [2]:
paper_pmid = 17660569
paper_name = 'wilson_vanhoof_2007' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [8]:
original_data = pd.read_excel('raw_data/hit_list.xlsx', sheet_name='Sheet1', skiprows=1)

In [9]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 17 x 6


In [10]:
original_data.head()

,Gene,Function/putative function,his3-nonstop,Steady state,Stability,dcp synthetic lethal
0,SKI2,Exosome-mediated mRNA degradation,++,ND,ND,ND
1,SKI3,Exosome-mediated mRNA degradation,++,2,2.7,+
2,SKI7,Exosome-mediated mRNA degradation,++,2.7,2.5,+
3,SKI8,Exosome-mediated mRNA degradation,++,2.1,2.5,+
4,PRE9,Proteasome-mediated protein degradation,++,0.4,ND,ND


In [11]:
original_data['gene'] = original_data['Gene'].astype(str)

In [12]:
# Eliminate all white spaces & capitalize
original_data['gene'] = clean_genename(original_data['gene'])

In [13]:
# Translate to ORFs 
original_data['orf'] = translate_sc(original_data['gene'], to='orf')

In [14]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orf'])
print(original_data.loc[~t,])

                  Gene Function/putative function his3-nonstop Steady state  \
index_input                                                                   
16           Wild type                          —            −            1   

            Stability dcp synthetic lethal      gene       orf  
index_input                                                     
16                0.9                    −  WILDTYPE  WILDTYPE  


In [15]:
original_data = original_data.loc[t,:]

In [16]:
original_data['data'] = original_data['his3-nonstop'].apply(lambda x: len(x.strip()))

In [17]:
original_data.set_index('orf', inplace=True)

In [18]:
original_data = original_data[['data']].copy()

In [19]:
original_data = original_data.groupby(original_data.index).mean()

In [20]:
original_data.shape

(16, 1)

# Load & process tested strains

In [21]:
tested = pd.read_excel('raw_data/deletion database.xlsx', sheet_name='Sheet1')

In [22]:
tested.head()

,Record #,ORF#,original plate location,Unnamed: 3,Unnamed: 4,Growth Phenotypes,Unnamed: 6
0,1830,YOR054c,BY4741,E,6,NaN,NaN
1,5714,YAL064C-A,chr00_1,A,2,NaN,NaN
2,5716,YBL091C-A,chr00_1,A,4,NaN,NaN
3,5717,YBR269C,chr00_1,A,5,NaN,NaN
4,5719,YBR271W,chr00_1,A,7,NaN,NaN


In [23]:
tested['orf'] = tested['ORF#'].astype(str)

In [24]:
tested['orf'] = clean_orf(tested['orf'])

In [27]:
tested.loc[tested['orf']=='YPL006','orf'] = 'YPL006W'
tested.loc[tested['orf']=='YDR007','orf'] = 'YDR007W'

In [28]:
tested['orf'] = translate_sc(tested['orf'], to='orf')

In [29]:
# Make sure everything translated ok
t = looks_like_orf(tested['orf'])
print(tested.loc[~t,])

Empty DataFrame
Columns: [Record #, ORF#, original plate location, Unnamed: 3, Unnamed: 4, Growth Phenotypes, Unnamed: 6, orf]
Index: []


In [30]:
tested_orfs = tested['orf'].unique()

In [31]:
missing = [orf for orf in original_data.index.values if orf not in tested_orfs]
missing

[]

In [32]:
original_data = original_data.reindex(index=tested_orfs, fill_value=0)

# Prepare the final dataset

In [33]:
data = original_data.copy()

In [34]:
dataset_ids = [169]
datasets = datasets.reindex(index=dataset_ids)

In [35]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [36]:
data.head()

dataset_id,169
data_type,value
orf,
YOR054C,0
YAL064C-A,0
YBL091C-A,0
YBR269C,0
YBR271W,0


## Subset to the genes currently in SGD

In [37]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 22


In [38]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,169
,data_type,value
gene_id,orf,
5267,YOR054C,0
1867,YAL064C-A,0
6108,YBL091C-A,0
463,YBR269C,0
465,YBR271W,0


# Normalize

In [39]:
data_norm = normalize_phenotypic_scores(data, has_tested=True)

In [40]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [41]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

dataset_id          169       
data_type         value valuez
gene_id orf                   
5267    YOR054C       0    0.0
1867    YAL064C-A     0    0.0
6108    YBL091C-A     0    0.0
463     YBR269C       0    0.0
465     YBR271W       0    0.0

# Print out

In [42]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [43]:
from IO.save_data_to_db3 import *

In [44]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/1 [00:00<?, ?it/s]

Deleting all datasets for PMID 17660569...
Inserting the new data...


100%|██████████| 1/1 [00:07<00:00,  7.86s/it]

Updating the data_modified_on field...
